## Simply average long period analysis
---

In [1]:
import numpy as np
import pandas as pd
from scripts import StockPriceDatabase

report = pd.read_csv("../database/reportData/Annual_Value.csv", index_col=[0])
report["Date"] = report["Date"].astype('datetime64[s]')
spd = StockPriceDatabase("../database/stockPrice/")
wig = pd.read_csv("../database/designData/WIGs.csv")

In [2]:
M_DAY_DISTANCE = 30

In [9]:
class StockPriceAnalyzer:
    @staticmethod
    def rate_of_return(stock_price: pd.DataFrame, 
                       date: np.datetime64,
                       value_type: str = "Close"):
        pass

    def avg_rate_of_return(stock_price: pd.DataFrame,
                           date: np.datetime64,
                           value_type: str = "Close"):
        pass

    def avg_stock_price(stock_price: pd.DataFrame,
                        date_begin: np.datetime64,
                        date_end: np.datetime64,
                        value_type: str = "Close"):
        
        pass

In [18]:
benchmarking_value = pd.DataFrame()

for idx, row in report.iterrows():
    date, ticker = row["Date"], row["Ticker"]
    # Company
    stock_price = spd.get_stock_price(row["Ticker"])
    stock_price.index = stock_price.index.astype('datetime64[s]')
    close_avg_earlier = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(-2*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(-1*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    close_avg_after   = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(1*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(2*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    company_growth_index = (close_avg_after - close_avg_earlier) / close_avg_earlier * 100
    
    # WIGs
    try:
        wig_ticker = np.where(wig.to_numpy() == row["Ticker"])[1][0]
    except:
        break
    stock_price = spd.get_stock_price(wig.columns[wig_ticker])
    stock_price.index = stock_price.index.astype('datetime64[s]')
    close_avg_earlier = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(-2*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(-1*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    close_avg_after   = stock_price[np.all([stock_price.index > row["Date"] + np.timedelta64(1*M_DAY_DISTANCE, 'D'),
                                            stock_price.index < row["Date"] + np.timedelta64(2*M_DAY_DISTANCE, 'D')], axis=0)]["Close"].mean()
    wig_growth_index = (close_avg_after - close_avg_earlier) / close_avg_earlier * 100

    benchmarking_against_index = company_growth_index - wig_growth_index
    
    if np.isnan(benchmarking_against_index):
        continue

    benchmarking_value = pd.concat([benchmarking_value, pd.DataFrame(data=[benchmarking_against_index], index=[idx])])

In [23]:
benchmarking_value.index[1:]benchmarking_value.index[0:-1]

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
      dtype='int64', length=1806)